# Data Collection 
## Steps for collecting our data:


 ## 1. 📸 Lectures images:
 We have taken our data from the database of the study lectures along the previous four years , which was about 5000 image of whiteboards.

## 2. Whiteboard Image Labeling with CVAT:

This notebook provides a complete guide to labeling whiteboard images using the Computer Vision Annotation Tool (CVAT) on your local machine.

## Prerequisites

Before we begin, ensure you have the following installed:

- Docker
- Docker Compose
- Git
- Python 3.6+

## 2.1 Setting Up CVAT Locally

### 2.1.1 Install Docker and Docker Compose

First, we need to install Docker on your local machine:

In [ ]:
# For Ubuntu/Debian:
!sudo apt update
!sudo apt install docker.io
!sudo systemctl start docker
!sudo systemctl enable docker

# Install Docker Compose
!sudo curl -L "https://github.com/docker/compose/releases/download/1.29.2/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
!sudo chmod +x /usr/local/bin/docker-compose

### 2.1.2 Clone and Setup CVAT

In [ ]:
# Clone the CVAT repository
!git clone https://github.com/opencv/cvat
%cd cvat

# Start CVAT containers
!docker-compose up -d

# Create superuser for CVAT (follow prompts)
!docker exec -it cvat bash -ic 'python3 ~/manage.py createsuperuser'

CVAT will now be running at `http://localhost:8080`. You can log in with the credentials you just created.

## 2.2 Preparing Your Data

### 2.2.1 Organize Your Images

Organize your whiteboard images in a directory structure like this:
```
whiteboard_data/
├── image_001.jpg
├── image_002.jpg
└── ...
```

### 2.2.2 Verify Image Integrity

Let's check our images before uploading (You can Skip the operation):

In [ ]:
import os
from PIL import Image
import pandas as pd

def check_images(directory_path):
    """
    Check all images in a directory for integrity and collect metadata
    """
    image_info = []
    corrupt_images = []
    
    for filename in os.listdir(directory_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            filepath = os.path.join(directory_path, filename)
            try:
                with Image.open(filepath) as img:
                    width, height = img.size
                    image_info.append({
                        'filename': filename,
                        'width': width,
                        'height': height,
                        'format': img.format,
                        'mode': img.mode,
                        'size_mb': os.path.getsize(filepath) / (1024 * 1024)
                    })
            except (IOError, OSError) as e:
                corrupt_images.append(filename)
                print(f"Corrupt image: {filename} - {str(e)}")
    
    return pd.DataFrame(image_info), corrupt_images

# Specify your image directory
image_dir = "whiteboard_data"
image_df, corrupt_list = check_images(image_dir)

print(f"Total images: {len(image_df)}")
print(f"Corrupt images: {len(corrupt_list)}")
image_df.head()

## 2.3 Creating a CVAT Project

### 2.3.1 Define Label Schema

First, we need to define what we're labeling on the whiteboards. Common labels might include:

- Text
- Diagrams
- Mathematical equations
- Tables
- Drawings

### 2.3.2 Create Project in CVAT UI

1. Open `http://localhost:8080` in your browser
2. Login with your credentials
3. Click "Create new project"
4. Name your project (e.g., "Whiteboard Analysis")
5. Add labels with appropriate names and colors

## 2.4 Creating Tasks and Uploading Images

### 2.4.1 Create Task in CVAT

1. In your project, click "Create new task"
2. Name your task (e.g., "Whiteboard Batch 1")
3. Select your project
4. Choose "Shared" if working with a team

### 2.4.2 Upload Images

You can upload images through the web interface or use the CVAT Python SDK or by simply just drag and drop into the local website.

## 2.5 Annotation Process

### 2.5.1 Using the CVAT Annotation Interface

1. Open your task in CVAT
2. Use the annotation tools:
   - Rectangle: For bounding boxes around elements
   - Polygon: For precise segmentation
   - Points: For keypoint annotation
   - Tag: For image-level labels

### 2.5.2 Keyboard Shortcuts

Learn these essential shortcuts for efficient labeling:

- `N` - Create a new shape
- `M` - Change mode (move, reshape, etc.)
- `F` - Move Forward to next Image.
- `D` - Move Backward to previous Image.
- `Tab` - Finish drawing a shape

### 2.5.3 Annotation Best Practices

- Be consistent with your labeling approach
- Use the same label for similar elements
- Ensure bounding boxes fully contain objects
- For overlapping objects, decide on a consistent approach

## 2.6. Quality Control

### 2.6.1 Review Process

After completing annotations, it's important to review them:

In [ ]:
def analyze_annotations(task_id):
    """
    Analyze annotations for consistency and completeness
    """
    with make_client(host="http://localhost:8080", credentials=("username", "password")) as client:
        task = client.tasks.retrieve(task_id)
        annotations = task.annotations
        
        # Get statistics
        stats = {}
        for label in task.labels:
            stats[label.name] = 0
        
        for shape in annotations.shapes:
            stats[shape.label] += 1
        
        return stats

# Get annotation statistics
task_id = 1  # Replace with your task ID
annotation_stats = analyze_annotations(task_id)
print("Annotation Statistics:")
for label, count in annotation_stats.items():
    print(f"{label}: {count} instances")

## 2.7 Exporting Annotations

### 2.7.1 Export Formats

CVAT supports multiple export formats:
- VOC XML
- COCO JSON
- YOLO
- TFRecord
- etc.


#### Note that CVAT only supports YOLO 1.1 export meaning you can only export unrotated boxes and not polygons or any other shape.

### 2.7.2 Export Through UI

1. Open your task
2. Click "Menu" → "Export task dataset"
3. Choose your preferred format
4. Download the annotations

### 2.7.3 Export Programmatically

In [ ]:
def export_annotations(task_id, format_name="COCO 1.0", output_path="annotations.json"):
    """
    Export annotations in specified format
    """
    with make_client(host="http://localhost:8080", credentials=("username", "password")) as client:
        task = client.tasks.retrieve(task_id)
        
        # Request export
        task.export_dataset(format_name, filename=output_path, include_images=False)
        
        # Download exported file
        task.download_export(output_path)
        
    print(f"Annotations exported to {output_path}")

# Export annotations
export_annotations(task_id=1, format_name="COCO 1.0", output_path="whiteboard_annotations.json")

## 2.8 Converting to Training Formats

### 2.8.1 Convert to YOLO Format

In [ ]:
import json
import os

def coco_to_yolo(coco_json_path, output_dir):
    """
    Convert COCO format annotations to YOLO format
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok)
    os.makedirs(os.path.join(output_dir, "labels"), exist_ok)
    
    # Load COCO data
    with open(coco_json_path) as f:
        data = json.load(f)
    
    # Create mapping from image id to image info
    images = {img['id']: img for img in data['images']}
    
    # Create mapping from category id to category name
    categories = {cat['id']: cat['name'] for cat in data['categories']}
    
    # Group annotations by image
    annotations_by_image = {}
    for ann in data['annotations']:
        image_id = ann['image_id']
        if image_id not in annotations_by_image:
            annotations_by_image[image_id] = []
        annotations_by_image[image_id].append(ann)
    
    # Process each image
    for image_id, annotations in annotations_by_image.items():
        image_info = images[image_id]
        image_width = image_info['width']
        image_height = image_info['height']
        filename = image_info['file_name'].split('.')[0]  # Remove extension
        
        # Create YOLO format file
        with open(os.path.join(output_dir, "labels", f"{filename}.txt"), "w") as f:
            for ann in annotations:
                category_id = ann['category_id']
                category_name = categories[category_id]
                
                # Convert bbox from [x, y, width, height] to YOLO format [center_x, center_y, width, height] (normalized)
                x, y, w, h = ann['bbox']
                center_x = (x + w/2) / image_width
                center_y = (y + h/2) / image_height
                norm_w = w / image_width
                norm_h = h / image_height
                
                # Write to file: class_id center_x center_y width height
                f.write(f"{category_id} {center_x:.6f} {center_y:.6f} {norm_w:.6f} {norm_h:.6f}\n")
    
    # Create classes file
    with open(os.path.join(output_dir, "classes.txt"), "w") as f:
        for cat_id, cat_name in sorted(categories.items()):
            f.write(f"{cat_name}\n")
    
    print(f"YOLO format annotations saved to {output_dir}")

# Convert COCO to YOLO
coco_to_yolo("whiteboard_annotations.json", "yolo_annotations")

# 3. 📊 Load data with their labels:
## YoloV8 data structure requirement:
Yolo library detection training models require a simplified data structure as shown





In [ ]:
whiteboard-Detection-Project/

├── images/
│   ├── train/              # Conatins Train images
│   └── val/            # Contains Validation Images
│             
├── Labels/ 
    ├── train/          # Contains Labels or annotation data of train images (has same name of the images to be matched and understood by YOLO)    
│   └── val/            # Contains Labels or annotation data of validiation images (has same name of the images to be matched and understood by YOLO)  
├── conf.yaml               
└── runs/               # Models which are trained are created here by YOLO at the end of training.



Yolo requires the paths of the train and validaition folders which is given inside conf.yaml along side the class-id names (.i.e in our case 0:whiteboards)
P.S
The Train and val images should be divided as 80% Train and 20% Validiations for optimal training.

In [ ]:
# Contents of conf.yaml
path: ./
train: ./data/images/train
val: ./data/images/val

names:
  0: Whiteboard

## 🤖 Yolov8 Training Code:
After having all the past steps setup, we can now run the training code provided below:

In [ ]:
import ultralytics
from ultralytics import YOLO

print(ultralytics.__version__)

model = YOLO("yolov8n.yaml") 
model.train(data="conf.yaml", epochs=100)



The past code simply uses yolov8 nano to train the data with 100 epochs, which we had used to train our first model (This method uses CPU)

## 💯 Training Yolo on the GPU using CUDA
Since Training on large data takes along time, using GPU for such processes could cut down the waiting time by much. This is the code we used:
We also switched the model to YoloV8 Small version instead of nano which has more accuarcy yet takes longer time to process.

In [ ]:
import torch
import ultralytics
from ultralytics import YOLO
from pathlib import Path
import os

# In order to navigate in project folder freely in code
current_dir = Path(__file__).parent
project_dir = current_dir.parent.parent
yolo_models_dir = project_dir / "src" / "data"


def main():
    print("Ultralytics version:", ultralytics.__version__)
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("GPU Name:", torch.cuda.get_device_name(0))

    # Load YOLO model
    model = YOLO(project_dir / "src" / "data" / "yolov8s.pt")

    # Train on GPU (device=0). Reduce workers to avoid multiprocessing issues.
    model.train(
        data= project_dir / "conf.yaml",
        epochs=200,
        device=0,
        workers=0,   # <--- IMPORTANT for Windows
        project = project_dir / "src" / "models" ,
        name = "Whiteboard Model"
    
    )


if __name__ == "__main__":
    main()

## 📑 Yolo Parameters we have used to train our latest model:

In [ ]:
from math import degrees
import torch
from torch.optim import AdamW
import ultralytics
from ultralytics import YOLO
from pathlib import Path


current_dir = Path(__file__).parent
project_dir = current_dir.parent.parent
yolo_models_dir = project_dir / "src" / "data"


def main():

    ## Checks Version of Yolo and the availablity of CUDA for training Via GPU
    print("Ultralytics version:", ultralytics.__version__)
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("GPU Name:", torch.cuda.get_device_name(0))

    # Load YOLO model
    model = YOLO(project_dir / "src" / "data" / "yolov8s.pt")

    # Train on GPU (device=0). Reduce workers to avoid multiprocessing issues.
    model.train(
        data="conf.yaml",
        epochs=70,
        device=0,
        workers=0,
        lr0=0.001,           # Initial learning rate
        lrf=0.01,            # Final learning rate
        degrees=3,          # Image rotation degrees
        optimizer="AdamW",   # Optimizer as string
        patience=15,         # Early stopping patience
        imgsz=640,           # Image size
        batch=16,            # Batch size
        project = project_dir / "src" / "models" ,
        name = "Whiteboard Model",            
        save=True,           # Save checkpoints
        plots=True           # Generate training plots
    )


if __name__ == "__main__":
    main()

# Automatic Labeling Feature:

We used an intelligent method to reduce the time and hardness of Labeling Process as follows:

1. Train a preliminary model on a subset of your data
2. Use it to pre-annotate the remaining images and save the images and their lables in a specfic folder named 'labeling_images' in "results" (i.e., .\results\labeling_images).
3. Manually correct the annotations

In [ ]:
from ultralytics import YOLO
import os
from pathlib import Path

current_dir = Path(__file__).parent
project_dir = current_dir.parent.parent

# Path to your trained model
MODEL_PATH = project_dir / "src" / "models" / "Whiteboard Model4" / "weights" / "best.pt"   # replace with your path

# Path to the folder containing images to label
IMAGE_FOLDER = project_dir / "src" / "data" / "image to label"     # replace with your folder path

# Path to the folder containing the labels files and images
RESULTS_FOLDER = project_dir / "results"     # replace with your folder path

# Confidence threshold for detection
CONF_THRESHOLD = 0.5 



def detect_whiteboards(input_dir, output_dir = RESULTS_FOLDER ):
                    
    # Load model
    model = YOLO(MODEL_PATH)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Run detection
    results = model.predict(
        source=input_dir,
        conf=CONF_THRESHOLD,
        save=True,
        project=output_dir,
        name='labeling_images',
        exist_ok=True,
        save_txt=True,
        #save_conf=True,
        #stream=True
    )
    
    print(f"Detection complete! Results saved to: {output_dir}")
    return results

# Usage
detect_whiteboards(IMAGE_FOLDER)

## Find WhiteBoard images:

The following python script is about finding whiteboard images from any generic images and soring them to a specific folder named "detected_images" in "results" folder (i.e., .\results\detected_images).

In [ ]:
import os
from ultralytics import YOLO
from pathlib import Path

current_dir = Path(__file__).parent
project_dir = current_dir.parent.parent

# Path to your trained model
MODEL_PATH = project_dir / "src" / "models" / "Whiteboard Model4" / "weights" / "best.pt"   # replace with your path

# Path to the folder containing images to check
IMAGE_FOLDER = project_dir / "src" / "data" / "images to test"     # replace with your folder path

# Path to the folder containing the labels files and images
RESULTS_FOLDER = project_dir / "results" / "detected_images"   # replace with your folder path

# Confidence threshold for detection
CONF_THRESHOLD = 0.5 


# Load your custom trained model
model = YOLO(MODEL_PATH)

os.makedirs(RESULTS_FOLDER, exist_ok=True)

# Run inference on all images in the source directory
results = model.predict(source=IMAGE_FOLDER, conf=CONF_THRESHOLD, save=False, stream=True)

# Get class names from the model
class_names = model.names
# Find the class ID for 'Whiteboard' (replace with your actual class name if different)
whiteboard_class_id = None
for idx, name in class_names.items():
    if name.lower() == 'whiteboard':
        whiteboard_class_id = idx
        break

if whiteboard_class_id is None:
    raise ValueError("Whiteboard class not found in model names.")

# Process results
for result in results:
    # Check if any detection is a whiteboard
    detections = result.boxes
    if detections is not None:
        class_ids = detections.cls.int().tolist()
        if whiteboard_class_id in class_ids:
            # Get the image path
            img_path = result.path
            # Generate output path
            output_path = os.path.join(RESULTS_FOLDER, os.path.basename(img_path))
            # Save the image (original or annotated)
            result.save(filename=output_path)  # Saves annotated image
            # Alternatively, to save the original image without annotations:
            # import shutil
            # shutil.copy(img_path, output_path)
            print(f"Saved image with whiteboard: {output_path}")

# YoloClassificationModelTest.py
### This code uses classification models instead of detection models to sort the read a folder full of images and classifiy them, then move the classified photos into a whiteboard folder.

In [ ]:
import os
import shutil
from pathlib import Path
from ultralytics import YOLO

def classify_and_organize_whiteboards(
    photos_folder, 
    model_path, 
    confidence_threshold=0.5,
    create_whiteboard_folder=True
):
    """
    Use a YOLO classification model to detect whiteboards in photos and organize them.
    
    Args:
        photos_folder (str): Path to folder containing photos to test
        model_path (str): Path to trained YOLO classification model (.pt file)
        confidence_threshold (float): Minimum confidence to classify as whiteboard (0.0-1.0)
        create_whiteboard_folder (bool): Whether to create a 'whiteboards' subfolder
    
    Returns:
        dict: Summary statistics of the classification and organization
    """
    # Convert to Path objects
    photos_path = Path(photos_folder)
    
    # Check if photos folder exists
    if not photos_path.exists():
        raise FileNotFoundError(f"Photos folder not found: {photos_folder}")
    
    # Check if model file exists
    if not Path(model_path).exists():
        raise FileNotFoundError(f"Model file not found: {model_path}")
    
    # Load YOLO classification model
    print(f"🔄 Loading YOLO classification model from: {model_path}")
    model = YOLO(model_path)
    
    # Create whiteboards folder if requested
    whiteboards_folder = None
    if create_whiteboard_folder:
        whiteboards_folder = photos_path / "whiteboards"
        whiteboards_folder.mkdir(exist_ok=True)
        print(f"📁 Created whiteboards folder: {whiteboards_folder}")
    
    # Supported image extensions
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif', '.webp'}
    
    # Find all image files
    image_files = []
    for file_path in photos_path.iterdir():
        if file_path.is_file() and file_path.suffix.lower() in image_extensions:
            image_files.append(file_path)
    
    print(f"🔍 Found {len(image_files)} image files to process")
    print(f"🎯 Confidence threshold: {confidence_threshold}")
    print("-" * 60)
    
    # Statistics
    whiteboard_count = 0
    non_whiteboard_count = 0
    error_count = 0
    whiteboard_files = []
    non_whiteboard_files = []
    error_files = []
    
    # Process each image
    for i, image_file in enumerate(image_files, 1):
        try:
            print(f"[{i}/{len(image_files)}] Processing: {image_file.name}")
            
            # Run classification
            results = model.predict(str(image_file), verbose=False)
            
            # Get prediction results
            probs = results[0].probs.data.cpu().numpy()
            class_names = results[0].names
            predicted_class = class_names[probs.argmax()]
            confidence = probs.max()
            
            print(f"   📊 Prediction: {predicted_class} (confidence: {confidence:.3f})")
            
            # Check if it's classified as whiteboard with sufficient confidence
            is_whiteboard = (predicted_class.lower() in ['whiteboard', 'whiteboards', '0'] and 
                           confidence >= confidence_threshold)
            
            if is_whiteboard:
                whiteboard_count += 1
                whiteboard_files.append(image_file.name)
                
                if create_whiteboard_folder:
                    # Move to whiteboards folder
                    dest_path = whiteboards_folder / image_file.name
                    shutil.move(str(image_file), str(dest_path))
                    print(f"   ✅ WHITEBOARD DETECTED → Moved to whiteboards folder")
                else:
                    print(f"   ✅ WHITEBOARD DETECTED → Keeping in place")
            else:
                non_whiteboard_count += 1
                non_whiteboard_files.append(image_file.name)
                print(f"   ❌ Not a whiteboard → Keeping in place")
                
        except Exception as e:
            error_count += 1
            error_files.append(image_file.name)
            print(f"   ⚠️  ERROR processing {image_file.name}: {str(e)}")
    
    # Print summary
    print("\n" + "=" * 60)
    print("📊 CLASSIFICATION SUMMARY")
    print("=" * 60)
    print(f"✅ Whiteboards detected: {whiteboard_count}")
    print(f"❌ Non-whiteboards: {non_whiteboard_count}")
    print(f"⚠️  Errors: {error_count}")
    print(f"📁 Total processed: {len(image_files)}")
    
    if create_whiteboard_folder and whiteboard_count > 0:
        print(f"📂 Whiteboards moved to: {whiteboards_folder}")
    
    # Show some examples
    if whiteboard_files:
        print(f"\n🎯 Sample whiteboard files detected:")
        for file in whiteboard_files[:5]:  # Show first 5
            print(f"   • {file}")
        if len(whiteboard_files) > 5:
            print(f"   ... and {len(whiteboard_files) - 5} more")
    
    return {
        "total_images": len(image_files),
        "whiteboard_count": whiteboard_count,
        "non_whiteboard_count": non_whiteboard_count,
        "error_count": error_count,
        "whiteboard_files": whiteboard_files,
        "non_whiteboard_files": non_whiteboard_files,
        "error_files": error_files,
        "whiteboards_folder": str(whiteboards_folder) if create_whiteboard_folder else None
    }

def main():
    """Main function to run the whiteboard classification and organization."""
    
    # Configuration - MODIFY THESE PATHS AS NEEDED
    photos_folder = "test3"  # Folder with photos to test
    model_path = "src/models/Whiteboard Model Classification5/weights/best.pt"  # Path to your trained model
    
    # Optional: You can also try other model paths if the above doesn't exist
    alternative_models = [
        "src/models/Whiteboard Model Classification2/weights/best.pt",
        "src/models/Whiteboard Model Classification3/weights/best.pt",
        "yolov8n-cls.pt"  # Pre-trained model as fallback
    ]
    
    # Check if primary model exists, try alternatives if not
    if not Path(model_path).exists():
        print(f"⚠️  Primary model not found: {model_path}")
        print("🔍 Trying alternative models...")
        
        for alt_model in alternative_models:
            if Path(alt_model).exists():
                model_path = alt_model
                print(f"✅ Using alternative model: {model_path}")
                break
        else:
            print("❌ No suitable model found!")
            return
    
    # Configuration parameters
    confidence_threshold = 0.6  # Adjust this value (0.0-1.0) - higher = more strict
    create_whiteboard_folder = True  # Set to False if you don't want to move files
    
    print("🚀 Starting YOLO Whiteboard Classification and Organization")
    print(f"📁 Photos folder: {photos_folder}")
    print(f"🤖 Model: {model_path}")
    print(f"🎯 Confidence threshold: {confidence_threshold}")
    print(f"📂 Create whiteboard folder: {create_whiteboard_folder}")
    print("=" * 60)
    
    try:
        results = classify_and_organize_whiteboards(
            photos_folder=photos_folder,
            model_path=model_path,
            confidence_threshold=confidence_threshold,
            create_whiteboard_folder=create_whiteboard_folder
        )
        
        # Final success message
        if results["whiteboard_count"] > 0:
            print(f"\n🎉 SUCCESS! Found and organized {results['whiteboard_count']} whiteboard images!")
        else:
            print(f"\n📝 No whiteboards detected with confidence >= {confidence_threshold}")
            
    except FileNotFoundError as e:
        print(f"❌ Error: {e}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")

if __name__ == "__main__":
    main()


# Labels-Converter.py
### A tool that helped in turning polygon annotations into simple boxes by approximating or estimating a box that covers the maximum x and y cooridnates of the polygon edges. This helped in annotating some small datasets that came with polygon annotations.

In [ ]:
import os

# Paths
labels_dir = "./Polygon_Labels"   # folder with your current polygon labels
output_dir = "./Box_Labels"      # folder for converted box labels
os.makedirs(output_dir, exist_ok=True)

for file in os.listdir(labels_dir):
    if not file.endswith(".txt"):
        continue

    input_path = os.path.join(labels_dir, file)
    output_path = os.path.join(output_dir, file)

    with open(input_path, "r") as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) <= 5:
            # Already a box label
            new_lines.append(line.strip())
            continue

        cls = parts[0]
        coords = list(map(float, parts[1:]))

        # Split into x,y pairs
        xs = coords[0::2]
        ys = coords[1::2]

        # Bounding box
        xmin, xmax = min(xs), max(xs)
        ymin, ymax = min(ys), max(ys)

        # Convert to YOLO box format
        x_center = (xmin + xmax) / 2
        y_center = (ymin + ymax) / 2
        width = xmax - xmin
        height = ymax - ymin

        new_line = f"{cls} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
        new_lines.append(new_line)

    # Save new file
    with open(output_path, "w") as f:
        f.write("\n".join(new_lines))

print("✅ Conversion complete! Box labels saved in:", output_dir)


# Labels and Images matching.py
### This code allowed for pulling of label files with same name of images from another folder, this helped when manual selection of and moving of labels was tedious in the process of data oragnizing for YOLO training.

In [ ]:
import os
import shutil
from pathlib import Path

def move_matching_text_files(images_folder, labels_folder):
    """
    Find all image files in the images folder, then search for matching text files
    in the labels folder and move them to the images folder.
    
    Supported image formats: .jpg, .jpeg, .png, .bmp, .tiff
    Text files should have the same base name as the image files.
    
    Args:
        images_folder (str): Path to folder containing image files
        labels_folder (str): Path to folder containing text label files
    
    Returns:
        dict: Summary of the operation with counts and file lists
    """
    # Convert to Path objects for easier handling
    images_path = Path(images_folder)
    labels_path = Path(labels_folder)
    
    # Check if folders exist
    if not images_path.exists():
        raise FileNotFoundError(f"Images folder not found: {images_folder}")
    if not labels_path.exists():
        raise FileNotFoundError(f"Labels folder not found: {labels_folder}")
    
    # Supported image extensions
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif'}
    
    # Find all image files
    image_files = []
    for file_path in images_path.iterdir():
        if file_path.is_file() and file_path.suffix.lower() in image_extensions:
            image_files.append(file_path)
    
    print(f"🔍 Found {len(image_files)} image files in {images_folder}")
    
    # Statistics
    moved_count = 0
    not_found_count = 0
    already_exists_count = 0
    moved_files = []
    not_found_files = []
    
    # Process each image file
    for image_file in image_files:
        # Get the base name without extension
        base_name = image_file.stem
        
        # Try multiple naming patterns for text files
        text_file = None
        
        # First try: exact match (Images (1234).txt)
        text_file = labels_path / f"{base_name}.txt"
        
        # Second try: convert "Images (1234)" -> "Image (1234)"
        if not text_file.exists() and base_name.startswith("Images ("):
            text_base_name = base_name.replace("Images (", "Image (", 1)
            text_file = labels_path / f"{text_base_name}.txt"
            
        # Third try: convert "Image (1234)" -> "Images (1234)" (reverse)
        if not text_file.exists() and base_name.startswith("Image ("):
            text_base_name = base_name.replace("Image (", "Images (", 1)
            text_file = labels_path / f"{text_base_name}.txt"
        
        if text_file.exists():
            # Check if text file already exists in images folder
            dest_text_file = images_path / f"{base_name}.txt"
            
            if dest_text_file.exists():
                print(f"⚠️  Text file already exists: {base_name}.txt")
                already_exists_count += 1
            else:
                # Move the text file to images folder
                shutil.move(str(text_file), str(dest_text_file))
                moved_count += 1
                moved_files.append(base_name)
                print(f"✅ Moved: {base_name}.txt")
        else:
            print(f"❌ No text file found for: {image_file.name}")
            not_found_count += 1
            not_found_files.append(image_file.name)
    
    # Print summary
    print(f"\n📊 Summary:")
    print(f"   ✅ Moved: {moved_count} text files")
    print(f"   ⚠️  Already existed: {already_exists_count} text files")
    print(f"   ❌ Not found: {not_found_count} text files")
    print(f"   📁 Total images processed: {len(image_files)}")
    
    return {
        "moved_count": moved_count,
        "already_exists_count": already_exists_count,
        "not_found_count": not_found_count,
        "total_images": len(image_files),
        "moved_files": moved_files,
        "not_found_files": not_found_files
    }

def main():
    """Main function to run the text file moving operation."""
    # Define the folders
    images_folder = "data/images/temp vald"
    labels_folder = "data/images/temp vald labels"
    
    print("🚀 Starting text file matching and moving process...")
    print(f"📁 Images folder: {images_folder}")
    print(f"📁 Labels folder: {labels_folder}")
    print("-" * 50)
    
    try:
        results = move_matching_text_files(images_folder, labels_folder)
        
        if results["moved_count"] > 0:
            print(f"\n🎉 Successfully moved {results['moved_count']} text files!")
        else:
            print("\n⚠️  No text files were moved.")
            
    except FileNotFoundError as e:
        print(f"❌ Error: {e}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")

if __name__ == "__main__":
    main()
